# Car Price Prediction

**Introduction**

* The objective of this kernel is to predict the car's price and understand how various factors would influence it. The website www.cardekho.com provided us with all necessary information on a wide variety of cars, including their selling prices. 

* I plan to use this information to find an optimal set of parameters that would allow us to predict the Selling Price of the car as accurately as possible. This is a regression problem, thus I would apply various regression models for the assessment of the relationship between independent variables and the dependent variable (Selling_Price) in this dataset.

* I have compared the most common regression models and found that KNN regressor (k-nearest neighbors) resulted in the highest r2 score of 0.83 

* Please do UpVote this Notebook if you find it helpful!

**Uploading data and libraries**

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')
df.head()

 <a></a>
# Table of contents


1. [Quick glance at Data](#Quick-glance-at-the-Data)

2. [My Hypothesis](#Hypothesis)

3. [Exploratory Data Analysis](#EDA)

4. [Feature Engineering](#Feature_Engineering)

5. [Feature Selection](#Feature_Selection)

6. [Model Building](#Model_Building)

7. [Hyperparameters Optimization](#Hyperparameters_Optimization)

8. [Model Evaluation](#Model_Evaluation)

9. [Deployment](#Deployment)





<a class = 'anchor' id = 'Quick-glance-at-the-Data'></a>

# Quick glance at Data

In [ ]:
# shape and data types of the data

print(df.dtypes)

print()
# select numeric columns
print('Numerical columns:')
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

# select non numeric columns
print('Categorical columns:')
df_cat = df.select_dtypes(exclude=[np.number])
cat = df_cat.columns.values
print(cat)

print()
print('Shape of this df is:')
print(df.shape)

In [ ]:
df.info()

In [ ]:
#Checking the null values
round(df.isnull().sum()/df.shape[0]*100,2)

no null values found in this dataset

In [ ]:
#Just a minor Feature Engeneering to get the Car's age:
df["Car_Age"] = 2021 - df['Year']
df.drop('Year',axis=1,inplace = True)

<a class = 'anchor' id = 'Hypothesis'></a>

# My Hypothesis

In [ ]:
df.head()

It is a good idea to have some ideas about the project before doing any exploratory data analysis:

**My hypothesis was that the car's Age (column Year) would influence the Selling Price at most**

<a class = 'anchor' id = 'Exploratory-Data-Analysis'></a>

<a class = 'anchor' id = 'EDA'></a>

# Exploratory Data Analysis




In [ ]:
#Car_Name column is not very usefull
df.drop('Car_Name', axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
#Frequency of each category separated by label
plt.figure(figsize=[15,18])
feafures = df.columns[1:]
#take all columns except the target Selling_Price
n=1
for f in feafures:
    plt.subplot(5,2,n)
    sns.barplot(x=f,  y= df['Selling_Price'], data=df, palette='twilight')
    plt.title("Countplot of {}  by Selling_Price".format(f))
    n=n+1
plt.tight_layout()
plt.show()

**Summary of EDA analysis**: how the independent features influence the dependent feature (Selling_Price) 

1. Present_Price: as Present_Price increases, Selling_Price increases as wel;, demonstrating a steep increase at the higher Present_Price values even though overall it shows a linear correlation with Selling_Price.
2. Kms_Driven: no correlation with Selling_Price was found.
3. Fuel_Type: cars with diesel is 3 times more expensive than those with Petrol and CNG.
4. Seller_Type: cars sold by dealer have higher Selling_Price than the individual sellers by 700% 
5. Transmission: automatic transmissions cost up to 2 times more than Manual ones!
6. Owner: used cars cost less than bran new, which is kind of common sense. 
6. Car_Age: unsurprisingly, old cars cost less than brand new ones.

Let's look at the corellation plot of all variables

To do that let's change the categorical variables to the one-hot-encoded, so we can calculate the correlation matrix.

In [ ]:
#getting the one hot encode variables
df = pd.get_dummies(data = df,drop_first=True) 

In [ ]:
df.head()

In [ ]:
#calculating the correlation matrix
plt.figure(figsize=(8, 10))
sns.heatmap(df.corr()[['Selling_Price']].sort_values(by='Selling_Price'), 
                    annot=True, 
                    cmap='coolwarm', 
                    vmin=-1,
                    vmax=1) 

**I have used the absolute cutoff value of 0.5 as a general approach to classify the variables importance based on the correlation matrix.**

1. Fuel_Type_Diesel and Present_Price (with more than +0.5) have considerable positive correlation with Selling_Price.

2. Seller_Type_Individual and Petrol_Type_Diesel (with less than -0.5) have significant negative correlation with Selling_Price.

3. Other features such as Transmission_Manual and Car_Age having correlation coefficient in the range  0 - 0.5 showed lesser correlation with Selling_Price and, thus will be dropped.


<a class = 'anchor' id = 'Feature_Selection'></a>

# Feature Selection 

For now, we have a rough idea of variables that are positively and negatively correlated with the target feature. To make final decision, which variables from highly correlated we will use for our model, we need to access the strength of the "true" correlation between predictors by using the Variance Inflation Factor (VIF), which can give us a cleart idea of how these features explain and discribe the "inflated" variance.

In [ ]:
# We see that there is a nulticollinearity between the selected independent variables
#We need to check the inflation factor to make sure we can use the features

#Lets drop the target dependent feature
X = df.drop('Selling_Price', axis = 1)

#Lets scale the features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_st = sc.fit_transform(X)
#X_test_std = sc_x.transform(X_test)


# Checking for the VIF values of the variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor


# Creating a dataframe that will contain the names of all the feature variables and their VIFs
vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X_train_st, i) for i in range(X_train_st.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

VIF must of each variable must be less than 5, or ideally less than 2, for the safety of predictions, [ref_here](https://towardsdatascience.com/multicollinearity-in-data-science-c5f6c0fe6edf). Thus we have to drop the most "Inflated" variable (Fuel_Type_Diesel) and assess the set of predictors (independent features) again.

In [ ]:
X = df.drop(columns = ['Fuel_Type_Diesel', 'Selling_Price'], axis = 1)


#Lets scale the features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_st = sc.fit_transform(X)
#X_test_std = sc_x.transform(X_test)


# Checking for the VIF values of the variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor


# Creating a dataframe that will contain the names of all the feature variables and their VIFs
vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X_train_st, i) for i in range(X_train_st.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Great! all features have VIF of less than 2. For now, we can drop the uncorrelated columns as Kms_Driven and those that have less than +0.5 and more than -0.5 according to the correlation matrix: 

I do not want to use many variables for the model because I want to train it to generalize better and not to overfit the test dataset

In [ ]:
df1 = df.drop(columns = ['Kms_Driven','Owner', 'Car_Age',  'Fuel_Type_Diesel', 'Transmission_Manual'])

So, now we have only three major predictors: Seller_Type, Fuel_Type, and Present_Price.
Let's check the distribution of these, so we can check for the outliers that might affect our model, for Selling_Price and Present_Price, as others are just frequency features.

In [ ]:
num_cols = ['Selling_Price','Present_Price']

for i in range(2):
    fig = plt.figure(figsize=[15,4])

    sns.boxplot(x=num_cols[i], data=df)

  
    
    plt.show()



<a class = 'anchor' id = 'Feature_Engineering'></a>


# Feature Engineering

We see that data is rightly skewed, thus we need to correct for the outliers on the right side of the boxplot. There are some general strategies for dealing with the outliers: 
1. Dropping (I do not like it because we lose some information that night be important). You can drop them if that makes perfect sense: like Age of a person that is more than 200 years old!
2. Imputing outliers to the right end of the distribution (That is what we will do)
3. Grouping the outliers in a separate dataset or creating a feature from the outliers that can be added to the dataset along with the other independent variables.

In [ ]:
#calculating the Interquartile range of Selling_Price
IQR=df1.Selling_Price.quantile(0.75)-df1.Selling_Price.quantile(0.25)

#calculating the borders of the normal distribution of Selling_Price
lower_bridge=df1['Selling_Price'].quantile(0.25)-(IQR*1.5)
upper_bridge=df1['Selling_Price'].quantile(0.75)+(IQR*1.5)

#Inpute the outliers with the max values that are present for the normal distribution of Selling_Price
df1.loc[df1['Selling_Price']>=upper_bridge,'Selling_Price']=upper_bridge

In [ ]:

#calculating the Interquartile range of Present_Price
IQR=df1.Present_Price.quantile(0.75)-df.Present_Price.quantile(0.25)

#calculating the borders of the normal distribution of Present_Price
lower_bridge=df1['Present_Price'].quantile(0.25)-(IQR*1.5)
upper_bridge=df1['Present_Price'].quantile(0.75)+(IQR*1.5)

#Inpute the outliers with the max values that are present for the normal distribution of Present_Price
df1.loc[df1['Present_Price']>=upper_bridge,'Present_Price']=upper_bridge

Lets check again whether we got the outliers out of our dist or not

In [ ]:
num_cols = ['Selling_Price','Present_Price']

for i in range(2):
    fig = plt.figure(figsize=[15,4])

    sns.boxplot(x=num_cols[i], data=df1)

  
    
    plt.show()

Our outliers are now at the right border of the upper quartile: around 6 for Selling_Price and  around 10 for Present_Price respectively.


Lets start the model builing with our dataset!

<a class = 'anchor' id = 'Model_Building'></a>

# Model Building 

In [ ]:
df1.head()

This reduced dataset(df1) we will use for the Selling_Price prediction

In [ ]:
# Separating target variable and its features
y = df1['Selling_Price']
X = df1.drop('Selling_Price',axis=1)

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print("x train: ",X_train.shape)
print("x test: ",X_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

We need to scale features because some of the regression algorithms as Linear, Lasso, Ridge, KNN require scaling

In [ ]:
#uploading Standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_st = sc.fit_transform(X_train)
X_test_st = sc.transform(X_test)

In [ ]:
#uploading all necessary models that we will apply for this dataset 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import  XGBRegressor

In [ ]:

#creating a dictionary of the models(estimators) 
estimators = {
    'Linear Regression': [LinearRegression()],
    'Lasso' :[Lasso()],
    'Ridge' :[Ridge()],
    'KNN' :[KNeighborsRegressor()],
    'Decision Tree' :[DecisionTreeRegressor()],
    'Ransom Forest' :[RandomForestRegressor()],
    'XG Boost': [XGBRegressor()],
}

#writing a function to fit models above to the train dataset
def mfit(estimators, X_train_st, y_train):
    for m in estimators:
        estimators[m][0].fit(X_train_st, y_train)
        print(m+' fitted')

mfit(estimators, X_train_st, y_train)

In [ ]:
import sklearn.metrics as metrics

#applying the fitted models to the test dataset
def mpredict(estimators, X_test_st, y_test):
    outcome = dict()
    for m in estimators:
        y_pred = estimators[m][0].predict(X_test_st)
        outcome[m] = [round(metrics.r2_score(y_test, y_pred), 2), 
                      metrics.mean_absolute_error(y_test, y_pred),
                     
                     metrics.mean_squared_error(y_test, y_pred),
                     np.sqrt(metrics.mean_squared_error(y_test, y_pred))]
    return outcome

outcome = mpredict(estimators, X_test_st, y_test)

In [ ]:
#printing the regression errors as metrics for the model evaluation
for m in outcome:
    print('------------------------'+m+'------------------------')
    print('R2 score', round(outcome[m][0],2))
    print('MAE', round(outcome[m][1],2))
    print('MSE', round(outcome[m][2],2))
    print('RMSE', round(outcome[m][3],2))


Among all model KNN method score not only the highest r2 values, but at the same time the lowest
[MAE, MSE, RMSE](https://towardsdatascience.com/a-common-mans-guide-to-mae-and-rmse-d5efcd238221)


Now , lets do the hyperparaemter tuning for this method

<a class = 'anchor' id = 'Hyperparameters_Optimization'></a>

# Hyperparameter Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
model = KNeighborsRegressor()

# Select an algorithm
algorithm = KNeighborsRegressor()

# Create 3 folds. We will use 3 fold cross validation
seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)


# Define our candidate hyperparameters
hp_candidates = [{'n_neighbors': [2,3,4,5,6,7,8,9,10], 'weights': ['uniform','distance']}]


# Search for best hyperparameters
grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
grid.fit(X_train_st, y_train)


# Get the results
print(grid.best_estimator_)
print(grid.best_params_)

As you can see, the best hyperparameters we get are {'n_neighbors': 8, 'weights': 'uniform'}. Thus,lets 
use these exact parameters for our KNN model and see how that would affect MAE, MSE, RMSE, and R2 score.

In [ ]:

KNN = KNeighborsRegressor(n_neighbors = 8, weights = 'uniform')
KNN.fit(X_train_st, y_train)

pred = KNN.predict(X_test_st)

print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print("R2 score =", round(metrics.r2_score(y_test, pred), 2))

Yes! We not only improved the R2 score from 0.79 to 0.83, but at the same time further minimized the errors

In [ ]:
#lest check how the predicted values(pred) are correlated with the sample data(y_test)

sns.distplot(y_test-pred)

Looks like the distribution of predictors strongly resembles the actual one of the test dataset.

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('default')
plt.style.use('ggplot')

fig, ax = plt.subplots(figsize=(7, 3.5))

ax.scatter(X_test_st[:,0], y_test, color='k', label='Regression model')
ax.scatter(X_train_st[:,0], y_train, edgecolor='k', facecolor='red', alpha=0.7, label='Sample data')
ax.set_ylabel('Selling_price', fontsize=14)
ax.set_xlabel('Factors = f(Present_Price, Fuel_Type_Petrol, Seller_Type_Individual)', fontsize=14)
ax.legend(facecolor='white', fontsize=11)

fig.tight_layout()

fig.tight_layout()

In [ ]:
df1.head()

<a class = 'anchor' id = 'Model_Evaluation'></a>

# Model_Evaluation


**There are four key assumptions that need to be tested for a linear regression model:**

1. **Linearity**: The expected value of the dependent variable is a linear function of each independent variable, holding the others fixed (note this does not restrict you to use a nonlinear transformation of the independent variables i.e. you can still model f(x) = ax² + bx + c, using both x² and x as predicting variables.

2. **Independence**: The errors (residuals of the fitted model) are independent of each other.

3. **Homoscedasticity** (constant variance): The variance of the errors is constant with respect to the predicting variables or the response.

4. **Normality**: The errors are generated from a Normal distribution (of unknown mean and variance, which can be estimated from the data). Note, this is not a necessary condition to perform linear regression unlike the top three above. However, without this assumption being satisfied, you cannot calculate the so-called ‘confidence’ or ‘prediction’ intervals easily as the well-known analytical expressions corresponding to Gaussian distribution cannot be used. [Ref](https://towardsdatascience.com/how-do-you-check-the-quality-of-your-regression-model-in-python-fa61759ff685)

In [ ]:
df1.head()

At first, from the final dataset (df1) we need to create a linear model and assess it with the statistical tools

In [ ]:
import statsmodels.formula.api as sm
model=sm.ols(formula = 'Selling_Price ~ Present_Price + Fuel_Type_Petrol + Seller_Type_Individual', 
             data=df1)
fitted = model.fit()
print(fitted.summary())

We can observe an R-squared value of 0.798 demonstrating a decent correlation of Present_Price, Fuel_Type_Petrol, Seller_Type_Individual with the Sellling_Price. The most important part of this table are p-values that show the importance of the individual predictors for the overall model corresponding to the significance level providing that 0,000 < 0.5.

In [ ]:
#Checking for the linearity 
for c in df1.columns[1:]:
    plt.figure(figsize=(8,5))
    plt.title("{} vs. \nSelling_Price".format(c),fontsize=16)
    plt.scatter(x=df1[c],y=df1['Selling_Price'],color='blue',edgecolor='k')
    plt.grid(True)
    plt.xlabel(c,fontsize=14)
    plt.ylabel('Selling_Price',fontsize=14)

    plt.show()

The model shows a linear relationship for the Present_Price and Selling_Price especially for the lowest prices in the range of 0-5 (Present_Price). As Present_Price increases, the variance of Selling_Price peakes dramatically especially at the highest Present_Price decreasing the quality of our linear model.

In [ ]:
#Checking for Normality
from statsmodels.graphics.gofplots import qqplot
fig=qqplot(fitted.resid_pearson,line='45',fit = 'True')
plt.xlabel('Theoretical quantiles')
plt.ylabel('Sample quantiles')
plt.show()

In [ ]:

#Checking for Normality
from scipy.stats import shapiro
_,p=shapiro(fitted.resid)
if p>0.05:
    print("The residuals seem to come from Gaussian process")
else:
    print("The normality assumption may not hold")
    
plt.figure(figsize=(8,5))
plt.hist(fitted.resid_pearson,bins=20,edgecolor='k')
plt.ylabel('Count',fontsize=15)
plt.xlabel('Normalized residuals',fontsize=15)
plt.title("Histogram of normalized residuals",fontsize=18)
plt.show()

The Normality assumption is violated as the left side of the Residual distribution does not follow the normal distribution, thus we can anticipate that the model might not predict well the higher values of Selling_Price.

In [ ]:
#Checking for Homoscedasticity
plt.figure(figsize=(8,5))
p=plt.scatter(x=fitted.fittedvalues,y=fitted.resid,edgecolor='k')
xmin=min(fitted.fittedvalues)
xmax = max(fitted.fittedvalues)
plt.hlines(y=0,xmin=xmin*0.9,xmax=xmax*1.1,color='red',linestyle='--',lw=3)
plt.xlabel("Fitted values",fontsize=15)
plt.ylabel("Residuals",fontsize=15)
plt.title("Fitted vs. residuals plot",fontsize=18)
plt.grid(True)
plt.show()

A good model displays the constant variance of the residuals vs their fitted values. In our case we have there major clusters of the residuals as displayed on the X-axis: 1st cluster (0-2), 2nd cluster (3-6), and 3rd cluster (6-12) meaning that the variance of Selling_Price is increasing with the values of predictors meaning that our model might not work well at the higher values of Selling_Price.

In [ ]:
#Autocorrelation
from statsmodels.stats.stattools import durbin_watson
durbinWatson = durbin_watson(fitted.resid)
print(durbinWatson)

The autocorrelation assumption is violated. The Durbin-Watson value must be outside of the 0-2 range
Values from 0 to 2 show positive autocorrelation (common in time series data).


**Conclusions:**

Cars Present_price is the most important factor for predicting the cars Selling_Price.

<a class = 'anchor' id = 'Deployment'></a>

# Deployment

[Deployment website](https://carpriceprediction1-api.herokuapp.com/)

All files necessary for the deployment can be found [here](https://github.com/valera2042/Cars-price-prediction).
Generally, to predict the car's Selling_Price you could convert your Present Price to the scale of this dataset's Present_Price (0.32 - 92.6) then follow the deployment link and recalculate the Selling_Price. Finally, convert Selling_Price (0.1 - 35) back to your Selling Price.


Dear Reader, thank you for your attention!